# A brief introduction to web APIs
## Data 765 tutoring

Scraping refers to two different but related tasks.

1. Pulling data from a standardized source such REST APIs or GraphQL APIs
2. Retrieving a raw website then preprocessing it to pull out the data you need

The first method is easier than the second due to standardization. An API (Application Programming Interface) has endpoints that designate the data to be returned. The data are usually returned in a very clean format, such as JSON, which makes preprocessing mostly trivial. Python often has libraries that encapsulate calls to certain REST and GraphQL APIs. Thus, you can usually just use a library, such as [Pushshift](https://github.com/pushshift/api) or the [PokéAPI](https://pokeapi.co/) library, to gather the data you need without worrying about handling timeouts, caching, or even errors (depending on the library).

The second method is needed when you have to scrape a website which doesn't have an API or that has a very limited API. APIs are straightforward but scraping a website directly is often painful. The trick here is to find the exact elements and classes with the required information. This usually involves poking around the HTML in a browser or text editor coupled with some luck and guess and check. APIs usually remain stable over time unlike websites. Finally, the wonderful, modern web is full of frivolous generated interactive websites that need more [ingenuity](https://www.selenium.dev/) to scrape. Finally, scraping websites in this manner is dubiously legal. Some sites explictly prohibit it which means you need to be a bit more, uh, creative if you truly want to scrape.

Data 765 is an introductory class for non-programmers. We'll focus primarily on the first case, but we'll tackle pulling data using raw requests rather than using a library specifically for the API.

## What is an API?

API is a general term that doesn't only refer to websites. APIs refer to interfaces that define actions. Libraries, such as [NumPy](https://numpy.org/), [Pandas](https://pandas.pydata.org/), [Bevy](https://bevyengine.org/), [Vulkan](https://www.vulkan.org/), and [Tokio](https://tokio.rs/) all implement well defined APIs. Browsing the documentation for each explains how to work with the libraries as well as common actions such as creating an empty array with a certain shape in NumPy or how to register a function as a system in Bevy. APIs encapsulate lower level processes. For example, NumPy wraps around fast, lower level [linear algebra libraries](https://en.wikipedia.org/wiki/Basic_Linear_Algebra_Subprograms) that leverage [SIMD](https://en.wikipedia.org/wiki/SIMD). The linear algebra libraries implement APIs as well.

APIs build up functionality to ease use. A light switch is an interface. The switch completes a circuit for current to flow through a light fixture. A bulb is required to complete the circuit. You don't need to know anything about the circuit. You only need to know that if you flip the switch the light should turn on. Likewise, you don't need to understand where the data in a web API is stored or whether they're delivered from a CDN.

# REST APIs

I'll primarily focus on REST APIs and `GET` requests. Web APIs provide a consistent interface to request information using plain URLs. REST APIs aren't _only_ for data, but to simplify the explanations I'll only focus on data (you know, since we're in data 🐱).

We can contrast web APIs to protocols. Protocols define an entire communication mechanism to exchange data structures and manage state. Protocols are lower level such as HTTP itself, Bluetooth, or the [Signal](https://signal.org/docs/) protocol.

Web APIs are higher level. Anything that can send a `GET` request and receive data back can access REST APIs. Browsers, libraries, tools like cURL, et cetera can all use REST APIs. We're going to access REST APIs via Python instead of a browser. Programatically accessing a web API allows us to facilitate automation and saving data.

Since REST APIs are simply URLs, we can open them with our browser without any special tools.

Here's the documentation for the [Steam reviews API](https://partner.steamgames.com/doc/store/getreviews). Steam is a platform and store for PC games.

The documentation lists a URL, `https://store.steampowered.com/appreviews/<appid>?json=1`, along with parameters that the web API requires. The primary parameter is `appid` which is the number assigned to each game or program on Steam. The `appid` is most easily found by visiting the Store page for a game. For example, here's [Doom II's page](https://store.steampowered.com/app/2300/DOOM_II/). The `appid` is 2300 which you can see from the URL (`app/2300`).

Using the API is simple after you find the `appid`. You simply fill in the URL: https://store.steampowered.com/appreviews/2300?json=1

(Click that link to see a working example!)

Adding more parameters is simple as well. Let's say we wanted each page to return 100 recent reviews. We just need to set the `num_per_page` parameter to `100` and `filter` to `all`. Parameters are specified after a question mark with an equal sign designating the argument. Adding more than parameter requires ampersands separating each. We already have the `json` parameter set, so our new URL is: https://store.steampowered.com/appreviews/2300?json=1&filter=recent&num_per_page=100

Click that too!

## Lists of web APIs
Public REST APIs exist for a wide range of topics. There are several GitHub repositories with lists of public web APIs. [Here's a good list](https://github.com/public-apis/public-apis).

# Python and Requests

[Requests](https://docs.python-requests.org/en/latest/) is one of Python's primary HTTP requests libraries. Requests is so popular that other programming languages have equivalents that implement similar APIs. My favorite programming language, Rust, has [reqwest](https://docs.rs/reqwest/), for example.

Libraries like `Requests` encapsulate raw sockets to provide a stable base for building applications that handle HTTP requests. `Requests` provides many quality of life features such as connection pooling, cookie jars, streaming downloads, et cetera. With that said, the [basic process](https://en.wikipedia.org/wiki/Hypertext_Transfer_Protocol#Request_syntax) of making a `GET` request with just sockets without worrying too much about errors is actually pretty simple.

In [ ]:
import socket
import ssl

resp_decoded = ""
# Connection: close is needed in the headers or else recv would block waiting for more bytes.
cat_req = b"GET /fact HTTP/1.1\r\nHost: catfact.ninja\r\nAccept: text/html\r\nConnection: close\r\nUser-Agent: JoshTest\r\n\r\n"

# Defaults to AF_INET and SOCK_STREAM
with socket.socket() as sock:
    hostname = "catfact.ninja"
    context = ssl.create_default_context()
    # Errors aren't handled for simplicity.
    sock.connect((hostname, 443))

    # Catfact uses HTTPS so I'll wrap the socket in SSL.
    with context.wrap_socket(sock, server_hostname=hostname) as ssock:
        ssock.sendall(cat_req)

        while buf := ssock.recv(1024):
            resp_decoded += buf.decode()

print(resp_decoded)

The idea is that we send a small string as bytes, `GET / HTTP/1.1\r\n\Host: HOSTNAME`, to an HTTP server to get back a response. The first backslash represents the path to retrieve specifically which in this case is `/fact`. The code above seems a bit more complicated than what I described because I also have to wrap the socket in SSL because almost every site uses HTTPS instead of unencrypted HTTP these days.

Ultimately, this basic process isn't too complicated, but handling errors, pooling connections, adding more headers, et cetera all vastly increase the difficulty. `Requests` manages all of that for us as well as the HTTP method string. In other words, the `Requests` API encapsulates the above code but with much more features.

Here's how simple it is to retrieve the same URL as above as well as deserialize the response into a `dict`.

In [3]:
import requests

catfact_resp = requests.get("https://catfact.ninja/fact").json()

print(catfact_resp["fact"])

Cats are the world's most popular pets, outnumbering dogs by as many as three to one
